<a href="https://colab.research.google.com/github/kelifi/prediction-boston-price/blob/main/house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra


# Loading DataSet

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Exploratory Data Analysis

In [ ]:
train.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
Utilities          0
dtype: int64

#Handling missing values

In [ ]:
train['PoolQC']=train['PoolQC'].fillna('None')
train['MiscFeature']=train['MiscFeature'].fillna('None')
train['Alley']=train['Alley'].fillna('None')
train['Fence']=train['Fence'].fillna('None')
train['FireplaceQu']=train['FireplaceQu'].fillna('None')


In [ ]:
train['LotFrontage']=train['LotFrontage'].fillna(train['LotFrontage'].mean())

In [ ]:
train['GarageCond']

0       TA
1       TA
2       TA
3       TA
4       TA
        ..
1455    TA
1456    TA
1457    TA
1458    TA
1459    TA
Name: GarageCond, Length: 1460, dtype: object

In [ ]:
L=[x for x in train if x.startswith('Garage')]


In [ ]:
for i in train[L].select_dtypes(exclude='object').columns:
  train[i]=train[i].fillna(0)
for i in train[L].select_dtypes(include='object').columns:
  train[i]=train[i].fillna('None')  

In [ ]:
L2=[y for y in train if y.startswith('Bsmt')]

In [ ]:
L2

['BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'BsmtFullBath',
 'BsmtHalfBath']

In [ ]:
for i in train[L2].select_dtypes(exclude='object'):
  train[i]=train[i].fillna(0)
for i in train[L2].select_dtypes(include='object'):
  train[i]=train[i].fillna('None')  

In [ ]:
L3=[s for s in train if s.startswith('MasVnr')]

In [ ]:
L3

['MasVnrType', 'MasVnrArea']

In [ ]:
train['MasVnrType']=train['MasVnrType'].fillna(train['MasVnrType'].mode()[0])
train['MasVnrArea']=train['MasVnrArea'].fillna(train['MasVnrArea'].mode()[0])

In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


# Input and target columns

In [ ]:
train1=train.copy()
target=train['SalePrice']
 


In [ ]:
x_train=train.iloc[:,:80].values

In [ ]:
x_train[1]

array([2, 20, 'RL', 80.0, 9600, 'Pave', 'None', 'Reg', 'Lvl', 'AllPub',
       'FR2', 'Gtl', 'Veenker', 'Feedr', 'Norm', '1Fam', '1Story', 6, 8,
       1976, 1976, 'Gable', 'CompShg', 'MetalSd', 'MetalSd', 'None', 0.0,
       'TA', 'TA', 'CBlock', 'Gd', 'TA', 'Gd', 'ALQ', 978, 'Unf', 0, 284,
       1262, 'GasA', 'Ex', 'Y', 'SBrkr', 1262, 0, 0, 1262, 0, 1, 2, 0, 3,
       1, 'TA', 6, 'Typ', 1, 'TA', 'Attchd', 1976.0, 'RFn', 2, 460, 'TA',
       'TA', 'Y', 298, 0, 0, 0, 0, 0, 'None', 'None', 'None', 0, 5, 2007,
       'WD', 'Normal'], dtype=object)

# Updating Existing Features

In [ ]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,Other,1,65.0,8450,Pave,None,Reg,1,AllPub,1,Gtl,CollgCr,1,1,1,2Story,7,5,2003,2003,1,1,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,1,...,Y,1,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,0,0,2,2008,1,1,208500
1,2,Other,1,80.0,9600,Pave,None,Reg,1,AllPub,0,Gtl,Other,0,1,1,1Story,6,8,1976,1976,1,1,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,1,...,Y,1,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,0,0,5,2007,1,1,181500
2,3,Other,1,68.0,11250,Pave,None,IR1,1,AllPub,1,Gtl,CollgCr,1,1,1,2Story,7,5,2001,2002,1,1,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,1,...,Y,1,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,0,0,9,2008,1,1,223500
3,4,Other,1,60.0,9550,Pave,None,IR1,1,AllPub,0,Gtl,Other,1,1,1,2Story,7,5,1915,1970,1,1,Wd Sdng,Other,None,0.0,TA,TA,Other,TA,Gd,No,ALQ,216,Unf,0,540,756,1,...,Y,1,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,0,0,2,2006,1,0,140000
4,5,Other,1,84.0,14260,Pave,None,IR1,1,AllPub,0,Gtl,Other,1,1,1,2Story,8,5,2000,2000,1,1,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,1,...,Y,1,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,0,0,12,2008,1,1,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,Other,1,62.0,7917,Pave,None,Reg,1,AllPub,1,Gtl,Gilbert,1,1,1,2Story,6,5,1999,2000,1,1,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,1,...,Y,1,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,None,None,0,0,8,2007,1,1,175000
1456,1457,Other,1,85.0,13175,Pave,None,Reg,1,AllPub,1,Gtl,NWAmes,1,1,1,1Story,6,6,1978,1988,1,1,Plywood,Plywood,Other,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,1,...,Y,1,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,None,MnPrv,0,0,2,2010,1,1,210000
1457,1458,Other,1,66.0,9042,Pave,None,Reg,1,AllPub,1,Gtl,Other,1,1,1,2Story,7,9,1941,2006,1,1,Other,Other,None,0.0,Ex,Gd,Other,TA,Gd,No,GLQ,275,Unf,0,877,1152,1,...,Y,1,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,None,GdPrv,1,2500,5,2010,1,1,266500
1458,1459,Other,1,68.0,9717,Pave,None,Reg,1,AllPub,1,Gtl,NAmes,1,1,1,1Story,5,6,1950,1996,0,1,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,1,...,Y,0,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,None,Attchd,1950.0,Unf,1,240,TA,TA,Y,366,0,112,0,0,0,None,None,0,0,4,2010,1,1,142125


In [ ]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,Other,1,65.0,8450,1,0,0,1,AllPub,1,0,CollgCr,1,1,1,2Story,7,5,2003,2003,1,1,VinylSd,VinylSd,BrkFace,196.0,3,2,PConc,4,3,1,6,706,1,0,150,856,1,...,1,1,856,854,0,1710,1,0,2,1,3,1,3,8,7,0,0,Attchd,2003.0,2,2,548,3,3,2,0,61,0,0,0,0,0,0,0,0,2,2008,1,1,208500
1,2,Other,1,80.0,9600,1,0,0,1,AllPub,0,0,Other,0,1,1,1Story,6,8,1976,1976,1,1,MetalSd,MetalSd,None,0.0,2,2,CBlock,4,3,4,5,978,1,0,284,1262,1,...,1,1,1262,0,0,1262,0,1,2,0,3,1,2,6,7,1,3,Attchd,1976.0,2,2,460,3,3,2,298,0,0,0,0,0,0,0,0,0,5,2007,1,1,181500
2,3,Other,1,68.0,11250,1,0,1,1,AllPub,1,0,CollgCr,1,1,1,2Story,7,5,2001,2002,1,1,VinylSd,VinylSd,BrkFace,162.0,3,2,PConc,4,3,2,6,486,1,0,434,920,1,...,1,1,920,866,0,1786,1,0,2,1,3,1,3,6,7,1,3,Attchd,2001.0,2,2,608,3,3,2,0,42,0,0,0,0,0,0,0,0,9,2008,1,1,223500
3,4,Other,1,60.0,9550,1,0,1,1,AllPub,0,0,Other,1,1,1,2Story,7,5,1915,1970,1,1,Wd Sdng,Other,None,0.0,2,2,Other,3,4,1,5,216,1,0,540,756,1,...,1,1,961,756,0,1717,1,0,1,0,3,1,3,7,7,1,4,Detchd,1998.0,1,3,642,3,3,2,0,35,272,0,0,0,0,0,0,0,2,2006,1,0,140000
4,5,Other,1,84.0,14260,1,0,1,1,AllPub,0,0,Other,1,1,1,2Story,8,5,2000,2000,1,1,VinylSd,VinylSd,BrkFace,350.0,3,2,PConc,4,3,3,6,655,1,0,490,1145,1,...,1,1,1145,1053,0,2198,1,0,2,1,4,1,3,9,7,1,3,Attchd,2000.0,2,3,836,3,3,2,192,84,0,0,0,0,0,0,0,0,12,2008,1,1,250000


In [ ]:
cat_attribs = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
num_attribs = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

In [ ]:
train[cat_attribs] = train[cat_attribs].astype(str)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
train[cat_attribs] = encoder.fit_transform(train[cat_attribs])

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
train[num_attribs] = imputer.fit_transform(train[num_attribs])

In [ ]:
train['BsmtCond'].mode()

<bound method Series.mode of 0       3.0
1       3.0
2       3.0
3       4.0
4       3.0
       ... 
1455    3.0
1456    3.0
1457    4.0
1458    3.0
1459    3.0
Name: BsmtCond, Length: 1460, dtype: float64>

In [ ]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,0.0,1.0,65.0,8450.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,7.0,4.0,2003.0,2003.0,1.0,1.0,4.0,4.0,0.0,196.0,2.0,2.0,2.0,3.0,3.0,1.0,6.0,706.0,1.0,0.0,150.0,856.0,1.0,...,1.0,1.0,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,2.0,8.0,6.0,0.0,0.0,0.0,2003.0,2.0,2.0,548.0,3.0,3.0,2.0,0.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,1.0,1.0,208500.0
1,2,0.0,1.0,80.0,9600.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,6.0,7.0,1976.0,1976.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,3.0,3.0,4.0,5.0,978.0,1.0,0.0,284.0,1262.0,1.0,...,1.0,1.0,1262.0,0.0,0.0,1262.0,0.0,1.0,2.0,0.0,3.0,1.0,1.0,6.0,6.0,1.0,3.0,0.0,1976.0,2.0,2.0,460.0,3.0,3.0,2.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,1.0,1.0,181500.0
2,3,0.0,1.0,68.0,11250.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,7.0,4.0,2001.0,2002.0,1.0,1.0,4.0,4.0,0.0,162.0,2.0,2.0,2.0,3.0,3.0,2.0,6.0,486.0,1.0,0.0,434.0,920.0,1.0,...,1.0,1.0,920.0,866.0,0.0,1786.0,1.0,0.0,2.0,1.0,3.0,1.0,2.0,6.0,6.0,1.0,3.0,0.0,2001.0,2.0,2.0,608.0,3.0,3.0,2.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,1.0,1.0,223500.0
3,4,0.0,1.0,60.0,9550.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,1.0,1.0,1.0,1.0,7.0,4.0,1915.0,1970.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,4.0,1.0,5.0,216.0,1.0,0.0,540.0,756.0,1.0,...,1.0,1.0,961.0,756.0,0.0,1717.0,1.0,0.0,1.0,0.0,3.0,1.0,2.0,7.0,6.0,1.0,4.0,1.0,1998.0,1.0,3.0,642.0,3.0,3.0,2.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2006.0,1.0,0.0,140000.0
4,5,0.0,1.0,84.0,14260.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,1.0,1.0,1.0,1.0,8.0,4.0,2000.0,2000.0,1.0,1.0,4.0,4.0,0.0,350.0,2.0,2.0,2.0,3.0,3.0,3.0,6.0,655.0,1.0,0.0,490.0,1145.0,1.0,...,1.0,1.0,1145.0,1053.0,0.0,2198.0,1.0,0.0,2.0,1.0,4.0,1.0,2.0,9.0,6.0,1.0,3.0,0.0,2000.0,2.0,3.0,836.0,3.0,3.0,2.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,1.0,1.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,0.0,1.0,62.0,7917.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,6.0,4.0,1999.0,2000.0,1.0,1.0,4.0,4.0,1.0,0.0,1.0,2.0,2.0,3.0,3.0,1.0,1.0,0.0,1.0,0.0,953.0,953.0,1.0,...,1.0,1.0,953.0,694.0,0.0,1647.0,0.0,0.0,2.0,1.0,3.0,1.0,1.0,7.0,6.0,1.0,3.0,0.0,1999.0,2.0,2.0,460.0,3.0,3.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,1.0,1.0,175000.0
1456,1457,0.0,1.0,85.0,13175.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,6.0,5.0,1978.0,1988.0,1.0,1.0,3.0,3.0,2.0,119.0,1.0,2.0,0.0,3.0,3.0,1.0,5.0,790.0,3.0,163.0,589.0,1542.0,1.0,...,1.0,1.0,2073.0,0.0,0.0,2073.0,1.0,0.0,2.0,0.0,3.0,1.0,1.0,7.0,5.0,2.0,3.0,0.0,1978.0,1.0,2.0,500.0,3.0,3.0,2.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,2010.0,1.0,1.0,210000.0
1457,1458,0.0,1.0,66.0,9042.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,1.0,1.0,1.0,7.0,8.0,1941.0,2006.0,1.0,1.0,2.0,2.0,1.0,0.0,3.0,3.0,1.0,2.0,4.0,1.0,6.0,275.0,1.0,0.0,877.0,1152.0,1.0,...,1.0,1.0,1188.0,1152.0,0.0,2340.0,0.0,0.0,2.0,0.0,4.0,1.0,2.0,9.0,6.0,2.0,4.

In [ ]:
X_train=train.drop(['SalePrice'],axis=1)
y_train=train['SalePrice']

In [ ]:
import xgboost
classifier=xgboost.XGBRegressor()

In [ ]:
import xgboost
regressor=xgboost.XGBRegressor()

In [ ]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [ ]:
regressor=xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
regressor.fit(X_train,y_train)

XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=900,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1, verbosity=1)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense( 50,activation='relu'))

# Adding the second hidden layer
classifier.add(Dense( 25,activation='relu'))

# Adding the third hidden layer
classifier.add(Dense( 50,activation='relu'))
# Adding the output layer
classifier.add(Dense( 1))

# Compiling the ANN
classifier.compile(loss='mse', optimizer='Adamax',metrics=['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train.values, y_train.values,validation_split=0.20, batch_size = 10, epochs = 100)

Epoch 1/100
117/117 [==============================] - 1s 3ms/step - loss: 37577785344.0000 - accuracy: 0.0000e+00 - val_loss: 36262252544.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
117/117 [==============================] - 0s 2ms/step - loss: 29239500800.0000 - accuracy: 0.0000e+00 - val_loss: 21001273344.0000 - val_accuracy: 0.0000e+00
Epoch 3/100
117/117 [==============================] - 0s 2ms/step - loss: 12337852416.0000 - accuracy: 0.0000e+00 - val_loss: 6188890624.0000 - val_accuracy: 0.0000e+00
Epoch 4/100
117/117 [==============================] - 0s 2ms/step - loss: 6765424640.0000 - accuracy: 0.0000e+00 - val_loss: 5377344000.0000 - val_accuracy: 0.0000e+00
Epoch 5/100
117/117 [==============================] - 0s 1ms/step - loss: 6150977024.0000 - accuracy: 0.0000e+00 - val_loss: 5185243136.0000 - val_accuracy: 0.0000e+00
Epoch 6/100
117/117 [==============================] - 0s 2ms/step - loss: 5856558592.0000 - accuracy: 0.0000e+00 - val_loss: 5100002304.0000 - val_ac